In [1]:
import os
os.chdir("..")

In [2]:
# FULL TEST
from thermo_final_exp import *
full_test()

In [2]:
import json
import numpy as np
import math
from typing import Dict, List, Tuple
import copy
from itertools import combinations
from collections import Counter

import logging
from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector, ConflictPredicator
from GridPatternBuilder import build_habit_groups

root_folder = os.path.join(DATA_ROOT, THERMO_ROOT)

test_projects = [
    "HS3301",
    # "HS3304",
    # "HS3307",
    # "HS3308",
    "HS3309",
    # "HS3310",
    # "HS3311",
    # "HS3313",
    # "HS5302",
    # "HS5303",
    # "HS5304",
    # "HS5305",
    "HS5309",
    # "HS5314",
]
# all_setpoints = [18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5]
# setpoint_mapping = {
#     "thermostat": {"off": 0, 0: "off"}
# }
# for idx, point in enumerate(all_setpoints):
#     state_full_name = "auto" + DEVICE_MULTI_STATE_SUFFIX + "#" + str(point)
#     setpoint_mapping["thermostat"][state_full_name] = idx + 1
#     setpoint_mapping["thermostat"][idx + 1] = state_full_name

setpoint_mapping = {
    "thermostat": thermo_state
}
time_step = 1
ctx_info = ContextAccessor({
        TIME_CTX: {
            "range" : (0, 24*60),
            "interval" : 60,
        },
        "OutTemp#NUM": {
            "range": (-25, 35),
            "interval": 5,
        },
        THERMO_MODE_CTX: {},
        # WEEKDAY_CTX: {
        #     "range": (0, 6),
        #     "interval": 1,
        # },
})
ccp_alpha = 5e-5
capacity = {
    "thermostat":0
}
BOOL_SIM = False
BOOL_UMASS= False
test_dates = generate_test_date(root_folder, test_projects, test_ratio = 0.4, true_random=False, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
grid_pattern_cfg = {
    # "time_delta" : timedelta(minutes=10),
    "context_info" : ctx_info,
    "alpha": ccp_alpha,
    "test_dates": test_dates,
    "device_state_map": setpoint_mapping,
    "time_delta": timedelta(minutes=time_step),
}

In [3]:
habit_groups = {}
grid_data = {}
for p in test_projects:
    grid_data[p] = test_umass(
                    root_folder = root_folder, 
                    test_project=p, 
                    ctx_info=ctx_info, 
                    grid_cfg = grid_pattern_cfg,
                    is_sim=BOOL_SIM, 
                    is_umass=BOOL_UMASS)

for p in test_projects:
    habit_groups[p] = build_habit_groups(grid_data[p], ccp_alpha)
c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print("Final predicted conflicts" + str({x:len(final_conflicts[x]) for x in final_conflicts}))

[18.0, 24.5]
[18.0, 24.5]
[18.0, 24.5]
[18.0, 24.5]
[18.0, 24.5]
Final predicted conflicts{'thermostat': 5}


In [4]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))
        # for g in d_groups:
        #     print(ctx_info.coor_box_to_range(g["box"][0] + g["box"][1]))
        #     print(g["dis"])

probs_i = [(x["prob"], i) for i, x in enumerate(final_conflicts["thermostat"])]
# print(sorted(probs_i))

# for home, groups in habit_groups.items():
#     for d, d_groups in groups.items():
#         for i, g in enumerate(d_groups):
#             for j in range(i+1, len(d_groups)):
#                 box_1 = g["box"][0] + g["box"][1]
#                 box_2 = d_groups[j]["box"][0] + d_groups[j]["box"][1]
#                 if does_intersect(box_1, box_2):
#                     print("!!!")
            

The habit groups found in HS3301
thermostat 2
The habit groups found in HS3309
thermostat 1
The habit groups found in HS5309
thermostat 1


In [5]:
import copy
# Make ground truth:

gt_ctx_info = ContextAccessor({
        TIME_CTX: {
            "range" : (0, 24*60),
            "interval" : 60,
        },
        "OutTemp#NUM": {
            "range": (-25, 35),
            "interval": 5,
        },
        THERMO_MODE_CTX: {},
        # WEEKDAY_CTX: {
        #     "range": (0, 6),
        #     "interval": 1,
        # },
})
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(root_folder, p, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
    device_events[p] = device_evts
gtconflict_cfg = {
    "context_info": gt_ctx_info,
    "capacity": capacity
}
# test_dates = {}

conflict_finder = GtConflictFinder(gtconflict_cfg)
gt_conflicts, test_state_cnt = conflict_finder.get_Gt_conflict(ctx_evts, device_events, test_dates)
print(len(gt_conflicts))

2607


In [6]:
conflict_device = {
    d:[]
    for d in capacity
}

conflict_state_device = {
    d:{}
    for d in capacity
}
MIN_TEST_OBS = 19
cnts = []
for c in gt_conflicts:
    d = c["device"]
    conflict_device[d].append(c)
    s = gt_ctx_info.get_coor_by_ctx(c["ctx"])
    users = frozenset(c["device_states"].keys())
    cnts.append(test_state_cnt[d][users][s])
    if test_state_cnt[d][users][s] < MIN_TEST_OBS:
        continue
    if s not in conflict_state_device[d]:
        conflict_state_device[d][s] = {}
    conflict_state_device[d][s][users] = conflict_state_device[d][s].get(users, 0) + 1

    # if conflict_state_device[d][s][users] > test_state_cnt[d][s]:
    #     print(conflict_state_device[d][s][users], test_state_cnt[d][s])
    #     print(users, d, c["cur_time"], s)
    
print({d:len(c) for d,c in conflict_device.items()})
print({d:len(c) for d,c in conflict_state_device.items()})
print(len(test_dates))
print(Counter(cnts))

{'thermostat': 2607}
{'thermostat': 47}
145
Counter({6.0: 155, 7.0: 116, 22.0: 114, 26.0: 110, 24.0: 108, 20.0: 102, 8.0: 97, 17.0: 96, 33.0: 89, 30.0: 82, 9.0: 81, 5.0: 81, 27.0: 78, 4.0: 78, 3.0: 78, 13.0: 75, 10.0: 72, 21.0: 69, 11.0: 68, 15.0: 58, 19.0: 57, 12.0: 55, 29.0: 52, 35.0: 51, 14.0: 48, 39.0: 46, 23.0: 44, 37.0: 43, 16.0: 40, 2.0: 39, 25.0: 36, 34.0: 35, 44.0: 34, 28.0: 31, 40.0: 31, 31.0: 27, 47.0: 26, 18.0: 20, 46.0: 18, 42.0: 17, 38.0: 15, 1.0: 14, 36.0: 12, 32.0: 9})


In [7]:
# print(conflict_state_device["thermostat"])

In [8]:
all_users = test_projects
all_devices = capacity.keys()
user_pairs = list(combinations(all_users, 2))
conflict_predicator = ConflictPredicator(ctx_info, final_conflicts)
exp_cnt = 0
exp_gt_c = 0
exp_result = {
    d: {
        frozenset(u_pair): [0,0,0,0,0] 
        for u_pair in user_pairs
    }
    for d in all_devices}
max_p = 0
all_state_cnt = 0
gt_probs = []
o_static = [[0,0], [0,0]]
for d in all_devices:
    for u_pair in user_pairs:
        u_pair_set = frozenset(u_pair)
        it = np.nditer(test_state_cnt[d][u_pair_set], flags=['multi_index'])
        for count in it:
            all_state_cnt += count
            if count < MIN_TEST_OBS:
                continue
            state = it.multi_index
            gt_prob = 0.
            if (d in conflict_state_device) and \
                (state in conflict_state_device[d]) and \
                (u_pair_set in conflict_state_device[d][state]):

                gt_prob =  float(conflict_state_device[d][state][frozenset(u_pair)]) / count
            gt_ctx_snapshot = gt_ctx_info.coor_to_snapshot(state)

            pred_prob = conflict_predicator.get_prob_conflict(ctx_info.get_coor_by_ctx(gt_ctx_snapshot), u_pair, d)
            exp_cnt += 1
            gt_probs.append(gt_prob)
            if gt_prob > 0:
                # if gt_prob > 0.5:
                #     print(gt_ctx_info.coor_to_snapshot(state).values(), gt_prob, pred_prob, u_pair)
                exp_gt_c += 1
                exp_result[d][u_pair_set][0] += abs(pred_prob - gt_prob)
                exp_result[d][u_pair_set][1] += 1
                exp_result[d][u_pair_set][4] += gt_prob
                if gt_prob > 1:
                    print(gt_prob, pred_prob, d, u_pair)
                # if abs(gt_prob - pred_prob) > 0.2:
                # print(pred_prob, gt_prob, gt_ctx_snapshot, count, u_pair)
                max_p = max(gt_prob, max_p)

            else:
                exp_result[d][u_pair_set][2] += pred_prob - gt_prob
                exp_result[d][u_pair_set][3] += 1
                # print(pred_prob, gt_prob, gt_ctx_snapshot, count, u_pair)
        avg_gt = sum(gt_probs) / len(gt_probs)
        errors = [abs(x-avg_gt) for x in gt_probs]
        for i, e in enumerate(errors):
            if gt_probs[i] > 0.:
                o_static[0][0] += e
                o_static[0][1] += 1
            else:
                o_static[1][0] += e
                o_static[1][1] += 1
        avg_error = str(sum(errors) / len(errors))
        gt_probs = []
        print("Optimal single prediction for users: {} is : {}".format(u_pair, str(sum(errors) / len(errors))))


o_acc = [[0., 0.], [0., 0.]]
o_zero = 0.
for d, result in exp_result.items():
    for u_pair in user_pairs:
        u_pair_set = frozenset(u_pair)
        r= result[u_pair_set]

        o_acc[0][0] += r[0]   #total conflict
        o_acc[1][0] += r[2]   #total non-conflict
        o_acc[0][1] += r[1]   #total conflict no.
        o_acc[1][1] += r[3]   #total non-conflict no.
        o_zero += r[4]     #total zero prediction

        # if r[1] > 0:
        #     r.append((r[0] + r[2]) / (r[1] + r[3]))
        # if r[1] > 0:
        #     r[0] = r[0] / r[1]
        #     r[4] = r[4] / r[1]
        # if r[3] > 0:
        #     r[2] = r[2] / r[3]
        # print(u_pair, r[0], r[2])
    
print("The no. conf {}, no. non-conf {}".format(o_acc[0][1], o_acc[1][1]))
print("The overall accuracy for conf is {}".format(o_acc[0][0] / o_acc[0][1]))
print("The overall accuracy for non-conf is {}".format(o_acc[1][0] / o_acc[1][1]))
print("The overall acc is {}".format((o_acc[0][0] + o_acc[1][0])/(o_acc[0][1] + o_acc[1][1])))

print("The overall zero for conf is {}".format(o_zero / o_acc[0][1]))
print("The overall zero for all is {}".format(o_zero / (o_acc[0][1] + o_acc[1][1])))

print("The no. conf {}, no. non-conf {}".format(o_static[0][1], o_static[1][1]))
print("Baseline, optimal single prediction for conf is {}".format(o_static[0][0] / o_static[0][1]))
print("Baseline, optimal single prediction for non-conf is {}".format(o_static[1][0] / o_static[1][1]))
print("The overall baseline is {}".format((o_static[0][0] + o_static[1][0])/(o_static[0][1] + o_static[1][1])))

    # print("Baseline approach, optimal single prediction: " + str(sum(errors) / len(errors)))

Optimal single prediction for users: ('HS3301', 'HS3309') is : 0.14507711616114405
Optimal single prediction for users: ('HS3301', 'HS5309') is : 0.16605314099485496
Optimal single prediction for users: ('HS3309', 'HS5309') is : 0.25676690740058405
The no. conf 102.0, no. non-conf 5.0
The overall accuracy for conf is 0.17725133529591666
The overall accuracy for non-conf is 0.3677133994662749
The overall acc is 0.18615143175247545
The overall zero for conf is 0.48506137545806205
The overall zero for all is 0.46239495604413394
The no. conf 102, no. non-conf 5
Baseline, optimal single prediction for conf is 0.17575247373930808
Baseline, optimal single prediction for non-conf is 0.39636554578778177
The overall baseline is 0.18606149579764797


In [4]:
from refit_alpha_test import *

ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            THERMO_MODE_CTX: {},
            "OutTemp#NUM": {
                "range": (-25, 35),
                "interval": 5,
            },
        })
gt_ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            THERMO_MODE_CTX: {},
            "OutTemp#NUM": {
                "range": (-25, 35),
                "interval": 5,
            },
        })


def alpha_nonLinear_generator(alpha_min, alpha_max, alpha_step):
    auc_values = []
    alpha = alpha_min
    while alpha <= alpha_max:
        yield alpha
        if alpha_step <= alpha / 10:
            alpha_step *= 10
        alpha += alpha_step

print("Run test accuracy for context " + str(ctx_info.get_all_ctx_ordered()))
alpha_generator = alpha_nonLinear_generator(1e-6, 0.5, 1e-6)
acc_values = test_acc_alpha(
    ctx_info = ctx_info, 
    gt_ctx_info = gt_ctx_info, 
    root_folder = root_folder,
    alpha_generator = alpha_generator, 
    device_mapping=setpoint_mapping,
    time_step = 1,
    test_projects = test_projects,
    capacity = capacity,
    acc_device = "thermostat")


Run test accuracy for context ['min_of_day#NUM', 'thermo_mode#CAT', 'OutTemp#NUM']
Finish alpha 1e-06, result (0.14025419512191048, 0.23260433384795917, 0.1445696221651838)
Finish alpha 2e-06, result (0.1384999113034074, 0.23519783727817475, 0.14301850597512553)
Finish alpha 3e-06, result (0.1373836317496004, 0.23519783727817475, 0.1419543890172908)
Finish alpha 4e-06, result (0.13773238648373826, 0.23230239115537538, 0.14215154558054374)
Finish alpha 4.9999999999999996e-06, result (0.14030149690070864, 0.24296179926899483, 0.14509870729175006)
Finish alpha 5.999999999999999e-06, result (0.13930587320104756, 0.24296179926899483, 0.14414960806403576)
Finish alpha 6.999999999999999e-06, result (0.14182950853053392, 0.24296179926899483, 0.1465553165089667)
Finish alpha 8e-06, result (0.1410115183521405, 0.23798082307200094, 0.14554279427362932)
Finish alpha 9e-06, result (0.14003623914040708, 0.23798082307200094, 0.1446130888568367)
Finish alpha 1e-05, result (0.14003623914040708, 0.23798